<a href="https://colab.research.google.com/github/Saish31/AIML/blob/main/Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformer import Transformer # this is the transformer.py file
import torch
import numpy as np

In [2]:
english_file = 'train.en'
hindi_file = 'train.hi'

# Generated this by filtering Appendix code

START_TOKEN   = '<START>'
PADDING_TOKEN = '<PAD>'
END_TOKEN     = '<END>'

hindi_vocabulary = [
    START_TOKEN,
    ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    ':', '<', '=', '>', '?', '@',
    '[', '\\', ']', '^', '_', '`',
    'ँ', 'ं', 'ः', 'ऽ', 'ॐ',
    'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ॠ', 'ऌ', 'ए', 'ऐ', 'ओ', 'औ',
    '्', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॄ', 'ॅ', 'े', 'ै', 'ॉ', 'ो', 'ौ',
    'क', 'ख', 'ग', 'घ', 'ङ',
    'च', 'छ', 'ज', 'झ', 'ञ',
    'ट', 'ठ', 'ड', 'ढ', 'ण',
    'त', 'थ', 'द', 'ध', 'न',
    'प', 'फ', 'ब', 'भ', 'म',
    'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह',
    '०', '१', '२', '३', '४', '५', '६', '७', '८', '९',
    '{', '|', '}', '~',
    PADDING_TOKEN,
    END_TOKEN
]

english_vocabulary = [
    START_TOKEN,
    ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    ':', '<', '=', '>', '?', '@',
    '[', '\\', ']', '^', '_', '`',
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
    'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
    'y', 'z',
    '{', '|', '}', '~',
    PADDING_TOKEN,
    END_TOKEN
]


In [3]:
index_to_hindi = {k:v for k,v in enumerate(hindi_vocabulary)}
hindi_to_index = {v:k for k,v in enumerate(hindi_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [5]:
with open(english_file, 'r') as file:
    english_sentences = file.readlines()
with open(hindi_file, 'r') as file:
    hindi_sentences = file.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = 200000
english_sentences = english_sentences[:TOTAL_SENTENCES]
hindi_sentences = hindi_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
hindi_sentences = [sentence.rstrip('\n') for sentence in hindi_sentences]

In [6]:
english_sentences[:10]

['in reply, pakistan got off to a solid start.',
 'the european union has seven principal decision-making bodies, its institutions: the european parliament, the european council, the council of the european union, the european commission, the court of justice of the european union, the european central bank and the european court of auditors.',
 'the congress leader represents sivaganga lok sabha segment from tamil nadu.',
 'prompt the user about connection attempts',
 'further, the minister announced that deposit insurance and credit guarantee corporation (dicgc) has been permitted to increase deposit insurance coverage to rs.',
 'therefore, brothers, be more diligent to make your calling and election sure. for if you do these things, you will never stumble.',
 'the review committee meeting chaired by the district judges will be attended by collectors, sps, superintendents of jails and secretary of district legal services authority.',
 'police is present on the spot.',
 'prime ministe

In [7]:
hindi_sentences[:10]

['जिसके जवाब में पाक ने अच्छी शुरुआत की थी.',
 'यूरोपीय संघ के महत्वपूर्ण संस्थानों में यूरोपियन कमीशन, यूरोपीय संसद, यूरोपीय संघ परिषद, यूरोपीय न्यायलय एवं यूरोपियन सेंट्रल बैंक इत्यादि शामिल हैं।',
 'कांग्रेस नेता तमिलनाडु से शिवगंगा लोकसभा क्षेत्र का प्रतिनिधित्व करते हैं.',
 'संबंधन प्रयास के बारे में उपयोक्ता को प्रांप्ट करें',
 'वित्त मंत्री ने घोषणा कि जमा बीमा और ऋण गारंटी निगम (डीआईसीजीसी) को जमा राशि बीमा का दायरा, जो इस समय 1 लाख रुपये है उसे बढ़ाकर प्रति जमाकर्ता 5 लाख रुपये करने की अनुमति प्रदान कर दी गई है।',
 'इस कारण हे भाइयों, अपने बुलाए जाने, और चुन लिये जाने को सिद्ध करने का भली भांति यत्न करते जाओ, क्योंकि यदि ऐसा करोगे, तो कभी भी ठोकर न खाओगे।',
 'समिति में जिला एवं सत्र न्यायाधीश, कलेक्टर, पुलिस अधीक्षक व जिला विधिक सेवा प्राधिकरण के सचिव शामिल रहते हैं।',
 'पुलिस मौके पर मौजूद है।',
 'जम्मू एवं कश्मीर विधानसभा के लिए दूसरे चरण के तहत होने वाले मतदान के लिए प्रधानमंत्री नरेंद्र मोदी भारतीय जनता पार्टी (भाजपा) के पक्ष में शुक्रवार को यहां रैलियां करेंगे।',
 'और 305

In [8]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length Hindi: {np.percentile([len(x) for x in hindi_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )

97th percentile length Hindi: 259.0
97th percentile length English: 267.0


In [9]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(hindi_sentences)):
    hindi_sentence, english_sentence = hindi_sentences[index], english_sentences[index]
    if is_valid_length(hindi_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(hindi_sentence, hindi_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(hindi_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 109299
Number of valid sentences: 24596


In [10]:
hindi_sentences = [hindi_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [11]:
hindi_sentences[:3]

['जिसके जवाब में पाक ने अच्छी शुरुआत की थी.',
 'कांग्रेस नेता तमिलनाडु से शिवगंगा लोकसभा क्षेत्र का प्रतिनिधित्व करते हैं.',
 'संबंधन प्रयास के बारे में उपयोक्ता को प्रांप्ट करें']

In [12]:
import torch

d_model = 512     # Embedding dimension
batch_size = 30     # Number of sentence pairs per training batch
ffn_hidden = 2048   # Inner-layer dimension of each Transformer feed-forward block
num_heads = 8       # Number of attention heads in each multi-head attention layer
drop_prob = 0.1      # Dropout probability applied after attention & in FFN
num_layers = 1      # Number of stacked encoder & decoder layers(can increase for better translation but it will take even longer for training)
max_sequence_length = 200   # Maximum sequence length the model can handle
hi_vocab_size = len(hindi_vocabulary)

transformer = Transformer(d_model,
                          ffn_hidden,
                          num_heads,
                          drop_prob,
                          num_layers,
                          max_sequence_length,
                          hi_vocab_size,
                          english_to_index,
                          hindi_to_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [13]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(71, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [14]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, hindi_sentences):
        self.english_sentences = english_sentences
        self.hindi_sentences = hindi_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.hindi_sentences[idx]

In [15]:
dataset = TextDataset(english_sentences, hindi_sentences)

In [16]:
len(dataset)

24596

In [17]:
dataset[1]

('the congress leader represents sivaganga lok sabha segment from tamil nadu.',
 'कांग्रेस नेता तमिलनाडु से शिवगंगा लोकसभा क्षेत्र का प्रतिनिधित्व करते हैं.')

In [18]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [19]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[('in reply, pakistan got off to a solid start.', 'the congress leader represents sivaganga lok sabha segment from tamil nadu.', 'prompt the user about connection attempts', 'all 176 passengers died in the incident.', 'not invited', 'coconut oil and lemon juice pack:', 'new delhi, 14th may, 2010', 'algeria military plane crash', 'up and down', 'probably a lot of you know the story of the two salesmen who went down to africa in the 1900s.', 'zenan, and hadashah, and migdal-gad,', 'but all was not over.', 'what is jcpoa?', 'this, in a way, is one of the challenges of how to interpret a really good idea.', 'director: shyam benegal', 'obtain loans from a bank or an individual.', 'are doing.', 'he is at present lodged in a prison in london.', 'in both conditions your doctor can prescribe you some analgesics or other medicines, or he can refer you to another specialist or surgeon.', 'fcra comes into force w.e.f may 2011', 'quality time', "you don 't say?", 'what would we do without sugar?', 

In [20]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=hindi_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [21]:
NEG_INFTY = -1e9

def create_masks(eng_batch, hi_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, hi_sentence_length = len(eng_batch[idx]), len(hi_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      hi_chars_to_padding_mask = np.arange(hi_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, hi_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, hi_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, hi_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [22]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, hi_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, hi_batch)
        optim.zero_grad()
        hi_predictions = transformer(eng_batch,
                                     hi_batch,
                                     encoder_self_attention_mask.to(device),
                                     decoder_self_attention_mask.to(device),
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(hi_batch, start_token=False, end_token=True)
        loss = criterian(
            hi_predictions.view(-1, hi_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == hindi_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Hindi Translation: {hi_batch[0]}")
            hi_sentence_predicted = torch.argmax(hi_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in hi_sentence_predicted:
              if idx == hindi_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_hindi[idx.item()]
            print(f"Hindi Prediction: {predicted_sentence}")


            transformer.eval()
            hi_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hi_sentence)
                predictions = transformer(eng_sentence,
                                          hi_sentence,
                                          encoder_self_attention_mask.to(device),
                                          decoder_self_attention_mask.to(device),
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_hindi[next_token_index]
                hi_sentence = (hi_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break

            print(f"Evaluation translation (should we go to the mall?) : {hi_sentence}")
            print("-------------------------------------------")

Epoch 0
Iteration 0 : 5.620936870574951
English: in reply, pakistan got off to a solid start.
Hindi Translation: जिसके जवाब में पाक ने अच्छी शुरुआत की थी.
Hindi Prediction: 8,8त-8#8888888888ॉ\ॉऽछ-छ8ऽॠ88888888ॉ88ऽै88नथॉ<PAD>ङ888ंदॉॉछॉछछछढछोछषेॉषॉ--छ#-##ढ,,ढ,~~,,,ॉ^,,<PAD>,,छछॉढ,ढॉढॉ,ॉ,ॉॉॉॉ१०,ॉॉढॉ,,,ॉ,ॉ,,,,छ१ॉॉॉॉ,,~,,ँॉ,5,ढछ|5ॉ,,,ॉ,ॉॉ~-ाषष%1,ॉॉ१ष,तढछ,ॉॉ३णण^<PAD>१त11ऽ1त|||ऽऽढऽऽॉ०ऽऽऽऽऽ,
Evaluation translation (should we go to the mall?) : ('                                                                                                                                                                   ाा                                   ',)
-------------------------------------------
Iteration 100 : 3.4670729637145996
English: similarly, infrastructure -
Hindi Translation: इसी तरह, बुनियादी सुविधाएँ,
Hindi Prediction: हकर                             र                                                                                                        ा                                    

In [25]:
transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  hi_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hi_sentence)
    predictions = transformer(eng_sentence,
                              hi_sentence,
                              encoder_self_attention_mask.to(device),
                              decoder_self_attention_mask.to(device),
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_hindi[next_token_index]
    hi_sentence = (hi_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return hi_sentence[0]

In [26]:
translation = translate("what should we do when the day starts?")
print(translation)
#क्या करना चाहिए दिन की शुरुआत में हमें?

क्या क्या क्या कि क्या क्या है?<END>


In [31]:
translation = translate("why did they do this?")
print(translation)
#ऐसा क्यों किया?

क्या क्या क्या है?<END>


In [33]:
translation = translate("click this")
print(translation)
#almost similar. it says press

प्रास<END>


In [38]:
translation = translate("my name is singh")
print(translation)

मैं कि बात किया हैं<END>


<h3>Insights</h3>

*   Using word-based or BPE based tokenizations may help. Also, we will get valid word (or BPE) units if we do so.
*   Make sure the training set has a large variety of sentences that are not just about one topic like "work" and "government".
*   Increase the number of encoder / decoder units for better translations. It was set to the minimum of 1 of each unit here.
  
Overall this model definitely learnt something even though the output may not be always accurate.
